In [ ]:
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from scipy import stats
from scipy.stats import norm
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV

In [ ]:
plt.rcParams['svg.fonttype'] = 'none'

SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def create_scatter_plot_genes(X, Y, x_title, y_title):
    
    scatter_df = pd.DataFrame({x_title: X, y_title: Y})
    
    fig, ax = plt.subplots(figsize=(6,6))
    sns.scatterplot(data=scatter_df, x=x_title, y=y_title, s=35, ax=ax)
    
    ax.set_xlabel(x_title)
    ax.set_ylabel(y_title)
    lims = [
        #np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        #np.max([ax.get_xlim(), ax.get_ylim()])  # max of both axes
        -0.3, 1.0
    ]
    plt.plot(lims, lims, 'k--', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    return fig

In [ ]:
def create_scatter_plot_systems(X, Y, H, x_title, y_title):
    
    scatter_df = pd.DataFrame({x_title: X, y_title: Y, 'hue': H})
    
    fig, ax = plt.subplots(figsize=(6,6))
    sns.scatterplot(data=scatter_df, x=x_title, y=y_title, hue='hue', s=35, ax=ax)
    
    ax.set_xlabel(x_title)
    ax.set_ylabel(y_title)
    lims = [
        #np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        #np.max([ax.get_xlim(), ax.get_ylim()])  # max of both axes
        -0.3, 1.0
    ]
    plt.plot(lims, lims, 'k--', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    return fig

In [ ]:
def create_histogram(data, x_title, y_title, h_range, fraction):
    
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111)
    
    data_size = len(data)
    class_size = int(data_size * fraction)
    x_sens = data[class_size - 1]
    x_resist = data[data_size - class_size]
    
    plt.hist(x=[data[i] for i in range(class_size)], bins=9, rwidth=0.85)
    plt.hist(x=[data[i] for i in range(data_size - class_size, data_size)], bins=9, rwidth=0.85)
    plt.hist(x=[data[i] for i in range(class_size, data_size - class_size)], bins=2, color='darkgrey', rwidth=0.85)
    
    mu, std = norm.fit(data)
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=2)
    
    plt.axvline(x_sens, linewidth=2, color='black')
    plt.axvline(x_resist, linewidth=2, color='black')
    
    ax.grid(False)
    ax.set_xlabel(x_title)
    ax.set_ylabel(y_title)
    plt.show()
    return fig

In [ ]:
def create_line_curve(X, Y, x_title, y_title, x_lim, y_lim):
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111)
    ax.plot(X, Y)
    ax.grid(False)
    ax.set_xlim(x_lim)
    ax.set_ylim(y_lim)
    ax.set_xlabel(x_title)
    ax.set_ylabel(y_title)
    plt.show()
    return fig

In [ ]:
def create_kaplan_meier(T1, T2, E1, E2, entity):
    
    fig, ax = plt.subplots(figsize=(5, 5))
    kmf_d = KaplanMeierFitter()
    
    kmf_d = kmf_d.fit(T1, event_observed=E1, label='Sensitive (n = ' + str(len(T1)) + ')')
    kmf_d.plot(ci_show=False, ax=ax)
    print('Median Survival (Sensitive): {:.1f}'.format(kmf_d.median_survival_time_))
    
    kmf_d = kmf_d.fit(T2, event_observed=E2, label='Resistant (n = ' + str(len(T2)) + ')')
    kmf_d.plot(ci_show=False, ax=ax)
    print('Median Survival (Resistant): {:.1f}'.format(kmf_d.median_survival_time_))
    
    #ax.set_xlim(0, 200)
    ax.set_ylim(0.0, 1.1)
    ax.set_xlabel('Overall Survival Months')
    ax.set_ylabel('Surviving Fractions')
    plt.title(entity + " Survival plot")
    plt.show()
    
    return fig

In [ ]:
def stratify_samples(genie_drug_data, drug, fraction, is_inverted=False):

    data_size = len(genie_drug_data)
    category_size = int(data_size * fraction)
    genie_drug_data.sort_values(by=drug, inplace=True, ignore_index=True)

    response_list = []
    for i in range(category_size):
        response = 1
        if is_inverted:
            response = 0
        sample_id = genie_drug_data.at[i, 'Sample_ID']
        temp_df = pd.DataFrame([[sample_id, response]], columns=['Sample_ID', 'response'])
        response_list.append(temp_df)
    
    for i in range(data_size - category_size, data_size):
        response = 0
        if is_inverted:
            response = 1
        sample_id = genie_drug_data.at[i, 'Sample_ID']
        temp_df = pd.DataFrame([[sample_id, response]], columns=['Sample_ID', 'response'])
        response_list.append(temp_df)

    response_df = pd.concat(response_list, axis=0, ignore_index=True)
    return response_df

In [ ]:
def create_survival_overall(response_df):

    survival_list = []
    for i, row in response_df.iterrows():
        if row['response'] == -1:
            continue
        survival = row['Overall_Survival_Months']
        status = row['Overall_Survival_Status']
        event = 1
        if 'LIVING' in status:
            event = 0
        temp_df = pd.DataFrame([[survival, row['response'], event]], columns=['Survival_Months', 'response', 'event'])
        survival_list.append(temp_df)

    survival_df = pd.concat(survival_list, axis=0, ignore_index=True)
    return survival_df

In [ ]:
def genie_analysis_majority(test_data, genie_drug_data, drug, fold_size=5, fraction=0.5, resp_th=4):
    
    response_df_list = []
    genie_drug_data[drug] = 0.0
    corr_sum = 0.0
    count = 0
    for k in range(1, fold_size+1):
        
        predict_data = np.loadtxt('../models/model_ctg_av_' + drug + '_auc_' + str(k) + '/predict_genie_428.txt')
        pred_df = pd.Series(predict_data, name='pred')
        pred_df = pd.concat([test_data, pred_df], axis=1)[['cell_line', 'smiles', 'pred']]

        for i, row in genie_drug_data.iterrows():
            sampleId = row['Sample_ID']
            p_auc = list(pred_df[pred_df['cell_line'] == sampleId]['pred'])[0]
            genie_drug_data.at[i, drug] = p_auc
        genie_drug_data.sort_values(by=drug, inplace=True, ignore_index=True)
        
        response_df = stratify_samples(genie_drug_data, drug, fraction)
        response_df_list.append(response_df)
    
    for i, df in enumerate(response_df_list):
        if i==0:
            all_response_df = df
        else:
            all_response_df = pd.merge(all_response_df, df, on=['Sample_ID'])
            all_response_df['response'] = all_response_df['response_x'] + all_response_df['response_y']
            all_response_df.drop(['response_x', 'response_y'], axis=1, inplace=True)
    
    for i, row in all_response_df.iterrows():
        val = -1
        if row['response'] >= resp_th:
            val = 1
        elif row['response'] <= (fold_size - resp_th):
            val = 0
        all_response_df.at[i, 'response'] = val
        
    merged_response_data = pd.merge(all_response_df, genie_drug_data, on='Sample_ID')
    survival_df = create_survival_overall(merged_response_data)
        
    T1 = list(survival_df[survival_df['response'] == 1]['Survival_Months'])
    T2 = list(survival_df[survival_df['response'] == 0]['Survival_Months'])
    E1 = list(survival_df[survival_df['response'] == 1]['event'])
    E2 = list(survival_df[survival_df['response'] == 0]['event'])
        
    print('p-value: {:.4f}'.format(logrank_test(durations_A=T1, durations_B=T2, event_observed_A=E1, event_observed_B=E2).p_value))
    km_fig = create_kaplan_meier(T1, T2, E1, E2, drug)

    return survival_df, km_fig

In [ ]:
def genie_analysis_gene(alteration_data, sample_map, genie_drug_data, title, is_resistant):
    
    response_list = []
    for s, i in sample_map.items():
        response = alteration_data[i]
        if is_resistant:
            response = not response
        temp_df = pd.DataFrame([[s, response]], columns=['Sample_ID', 'response'])
        response_list.append(temp_df)

    response_df = pd.concat(response_list, axis=0, ignore_index=True)
    merged_response_data = pd.merge(response_df, genie_drug_data, on='Sample_ID')
    survival_df = create_survival_overall(merged_response_data)
        
    T1 = list(survival_df[survival_df['response'] == 1]['Survival_Months'])
    T2 = list(survival_df[survival_df['response'] == 0]['Survival_Months'])
    E1 = list(survival_df[survival_df['response'] == 1]['event'])
    E2 = list(survival_df[survival_df['response'] == 0]['event'])
        
    print('p-value: {:.4f}'.format(logrank_test(durations_A=T1, durations_B=T2, event_observed_A=E1, event_observed_B=E2).p_value))
    km_fig = create_kaplan_meier(T1, T2, E1, E2, title)
    
    return survival_df, km_fig

In [ ]:
def genie_analysis_system(system_name, test_data, genie_drug_data, drug, fold_size=5, fraction=0.5, resp_th=4):
    
    response_df_list = []
    genie_drug_data[drug] = 0.0
    corr_sum = 0.0
    count = 0
    for k in range(1, fold_size+1):
        
        predict_data = np.loadtxt('../models/model_ctg_av_' + drug + '_auc_' + str(k) + '/predict_genie_428.txt')
        pred_df = pd.Series(predict_data, name='pred')
        
        embedding_data = np.loadtxt('../models/model_ctg_av_' + drug + '_auc_' + str(k) + '/hidden_genie_428/' + system_name + '.hidden')
        pca = PCA(n_components=1)
        system_activity = pca.fit_transform(embedding_data)
        pca_df = pd.Series(system_activity[:, 0], name='pc1')
        
        merged_df = pd.concat([test_data, pred_df, pca_df], axis=1)[['cell_line', 'smiles', 'pred', 'pc1']]

        print('Fold:' , str(k), 'Spearman rho:', stats.spearmanr(merged_df['pred'], merged_df['pc1']))

        for i, row in genie_drug_data.iterrows():
            sampleId = row['Sample_ID']
            pc1 = list(merged_df[merged_df['cell_line'] == sampleId]['pc1'])[0]
            genie_drug_data.at[i, drug] = pc1
        genie_drug_data.sort_values(by=drug, inplace=True, ignore_index=True)
        
        is_inverted = False
        if stats.spearmanr(merged_df['pred'], merged_df['pc1'])[0] < 0:
            is_inverted = True
        response_df = stratify_samples(genie_drug_data, drug, fraction, is_inverted)
        response_df_list.append(response_df)
    
    for i, df in enumerate(response_df_list):
        if i==0:
            all_response_df = df
        else:
            all_response_df = pd.merge(all_response_df, df, on=['Sample_ID'])
            all_response_df['response'] = all_response_df['response_x'] + all_response_df['response_y']
            all_response_df.drop(['response_x', 'response_y'], axis=1, inplace=True)
    
    for i, row in all_response_df.iterrows():
        val = -1
        if row['response'] >= resp_th:
            val = 1
        elif row['response'] <= (fold_size - resp_th):
            val = 0
        all_response_df.at[i, 'response'] = val
        
    merged_response_data = pd.merge(all_response_df, genie_drug_data, on='Sample_ID')
    survival_df = create_survival_overall(merged_response_data)
        
    T1 = list(survival_df[survival_df['response'] == 1]['Survival_Months'])
    T2 = list(survival_df[survival_df['response'] == 0]['Survival_Months'])
    E1 = list(survival_df[survival_df['response'] == 1]['event'])
    E2 = list(survival_df[survival_df['response'] == 0]['event'])
        
    print('p-value: {:.4f}'.format(logrank_test(durations_A=T1, durations_B=T2, event_observed_A=E1, event_observed_B=E2).p_value))
    km_fig = create_kaplan_meier(T1, T2, E1, E2, system_name)

    return survival_df, km_fig

In [ ]:
def gene_importance(drug, cell_features, genie_features, gene_list, fold_size=5):
    
    all_cell_coef = np.zeros(len(gene_list))
    all_genie_coef = np.zeros(len(gene_list))
    for i in range(1, fold_size+1):
        
        modeldir = '../models/model_ctg_av_' + drug + '_auc_' + str(i)
        
        y_cell = np.loadtxt(modeldir + '/predict.txt')
        cell_test_df = pd.read_csv("../data/training_files_av/" + str(i) + "_test_av_" + drug + ".txt", sep='\t', header=None, names=['cell', 's', 'auc', 'd'])
        cell_lines_df = pd.read_csv('../data/training_files_av/cell2ind_av.txt', header=None, sep='\t', names=['I', 'C'])
        cell_sample_map = dict(zip(cell_lines_df.C, cell_lines_df.I))
        
        X_cell = np.empty(shape = (len(cell_test_df), len(cell_features[0, :])))
        for i, row in cell_test_df.iterrows():
            X_cell[i] = cell_features[int(cell_sample_map[row['cell']])]
            
        regr = RidgeCV(cv=5)
        regr.fit(X_cell, y_cell)
        cell_coef_df = pd.DataFrame(regr.coef_, index=gene_list, columns=['cell_coef'])
        cell_coef_df.sort_index()
        all_cell_coef += np.array(cell_coef_df['cell_coef'])

        y_genie = np.loadtxt(modeldir + '/predict_genie_428.txt')
        genie_samples = list(pd.read_csv('../data/GENIE/cell2ind_428.txt', header=None, sep='\t', names=['I', 'C'])['I'])
        genie_features_df = pd.DataFrame(genie_features, columns=gene_list, index=genie_samples)
        X_genie = genie_features_df.loc[genie_samples, gene_list]
        
        regr = RidgeCV(cv=5)
        regr.fit(X_genie, y_genie)
        genie_coef_df = pd.DataFrame(regr.coef_, index=gene_list, columns=['genie_coef'])
        genie_coef_df.sort_index()
        all_genie_coef += np.array(genie_coef_df['genie_coef'])
        
    all_cell_coef /= fold_size
    all_genie_coef /= fold_size
    fig = create_scatter_plot_genes(all_genie_coef, all_cell_coef, 'Gene Importance\n(Clinical samples)', 'Gene Importance\n(Cell lines)')
    print(stats.spearmanr(all_genie_coef, all_cell_coef)[0])
        
    return fig

In [ ]:
def create_gene_rho(drug, gene_list, data_type, fold_size=5):
    
    for i in range(1, fold_size+1):
        modeldir = '../models/Test/model_ctg_av_' + drug + '_auc_' + str(i)
        pred = np.loadtxt(modeldir + '/predict' + data_type + '.txt')
        hiddendir = modeldir + '/hidden' + data_type
        
        outf = open(modeldir + '/gene_rho' + data_type + '.txt', "w")
        outf.write('Gene\tRho\tP_val\n')
        for gene in gene_list:
            gene_embedding = np.loadtxt(hiddendir + '/' + gene + '.hidden')
            rho, p_val = stats.spearmanr(pred, gene_embedding)
            result = '{}\t{:.3f}\t{:.3e}\n'.format(gene, rho, p_val)
            outf.write(result)
        outf.close()

In [ ]:
def aggregate_gene_rho(drug, gene_list, data_type, fold_size=5):
    agg_gene_rho = []
    for i in range(1, fold_size+1):
        modeldir = '../models/Test/model_ctg_av_' + drug + '_auc_' + str(i)
        gene_rho_df = pd.read_csv(modeldir + '/gene_rho' + data_type + '.txt', sep='\t')[['Gene', 'Rho']]
        agg_gene_rho.append(gene_rho_df)
    agg_df = pd.concat(agg_gene_rho, ignore_index=True)
    agg_rho_df = pd.DataFrame(agg_df.groupby(['Gene']).mean())
    agg_rho_df.fillna(0, inplace=True)
    agg_rho_df.to_csv('../models/rlipp/' + drug + '_all_gene_rho' + data_type + '.txt', sep='\t', float_format='%.3f', index=True)

In [ ]:
func_map = {}
func_map['Palbociclib'] = 'CDK4_6_Inhibitor_Overall'

smiles_map = {'Palbociclib':"CC1=C(C(=O)N(C2=NC(=NC=C12)NC3=NC=C(C=C3)N4CCNCC4)C5CCCC5)C(=O)C"}

In [ ]:
#Common data

genie_data = pd.read_csv('../data/GENIE/brca_akt1_genie_2019_clinical_data.tsv', sep='\t')
genie_data.columns = genie_data.columns.str.replace(' ','_', regex=False)
genie_data.columns = genie_data.columns.str.replace('/','_', regex=False)
genie_data.columns = genie_data.columns.str.replace('(','', regex=False)
genie_data.columns = genie_data.columns.str.replace(')','', regex=False)

gene_list = list(pd.read_csv('../data/training_files_av/gene2ind_ctg_av.txt', sep='\t', header=None, names=(['I', 'G']))['G'])

sample_df = pd.read_csv('../data/GENIE/cell2ind_428.txt', header=None, sep='\t', names=['I', 'C'])
sample_list = list(sample_df['C'])
sample_map = dict(zip(sample_df.C, sample_df.I))

In [ ]:
drug = 'Palbociclib'

test_data = pd.read_csv("../data/GENIE/test_428_" + drug + ".txt", sep='\t', header=None, names=['cell_line', 'smiles', 'auc', 'dataset'])
    
genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_palbo = genie_analysis_majority(test_data, genie_drug_data, drug, fraction=0.5, resp_th=4)

In [ ]:
km_fig_palbo.savefig('../plots/figure6/survival_' + drug + '.svg')

In [ ]:
drug = 'Palbociclib'

test_data = pd.read_csv("../data/GENIE/test_428_" + drug + ".txt", sep='\t', header=None, names=['cell_line', 'smiles', 'auc', 'dataset'])
    
genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']

pos_samples = list(genie_drug_data['Sample_ID'])
genie_non_drug_data = genie_data.query("Sample_ID not in @pos_samples").copy()
print('Sample size:', len(genie_non_drug_data))
genie_non_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_non_palbo = genie_analysis_majority(test_data, genie_non_drug_data, drug, fraction=0.5, resp_th=4)

In [ ]:
km_fig_non_palbo.savefig('../plots/figure6/survival_non_' + drug + '.svg')

In [ ]:
cl_rlipp_df = pd.read_csv('../models/rlipp/' + drug + '_all_pval.txt', sep='\t')[['Term', 'Name', 'P_rho_mean']]
genie_rlipp_df = pd.read_csv('../models/rlipp/' + drug + '_all_pval_genie_428.txt', sep='\t')[['Term', 'Name', 'P_rho_mean']]

merged_df = pd.merge(cl_rlipp_df, genie_rlipp_df, on=['Term', 'Name'], suffixes=('_cl', '_genie'))
merged_df.fillna(0, inplace=True)

merged_df['hue'] = ''
for i, row in merged_df.iterrows():
    genie = row['P_rho_mean_genie']
    cl = row['P_rho_mean_cl']
    if genie < 0.3 and cl < 0.3:
        merged_df.at[i, 'hue'] = 'none'
    elif genie/cl >= 0.66 and genie/cl <= 1.5:
        merged_df.at[i, 'hue'] = 'genie_cell'
    elif genie/cl < 0.66:
        merged_df.at[i, 'hue'] = 'cell'
    elif genie/cl > 1.5:
        merged_df.at[i, 'hue'] = 'genie'

scatterplot_sys = create_scatter_plot_systems(merged_df['P_rho_mean_genie'], merged_df['P_rho_mean_cl'], merged_df['hue'],
                                      'System Importance\n(Clinical samples)', 'System Importance\n(Cell lines)')
print('Spearman rho:', stats.spearmanr(merged_df['P_rho_mean_cl'], merged_df['P_rho_mean_genie'])[0])

In [ ]:
scatterplot_sys.savefig('../plots/figure6/scatterplot_system_importance_hue_all_March_28.svg')

In [ ]:
cl_go_rlipp_df = pd.read_csv('../models/rlipp/' + drug + '_go_pval.txt', sep='\t')[['Term', 'P_rho_mean']]
genie_go_rlipp_df = pd.read_csv('../models/rlipp/' + drug + '_go_pval_genie_428.txt', sep='\t')[['Term', 'P_rho_mean']]

merged_go_df = pd.merge(cl_go_rlipp_df, genie_go_rlipp_df, on=['Term'], suffixes=('_cl', '_genie'))
merged_go_df.dropna(inplace=True)
scatterplot_sys_go = create_scatter_plot_systems(merged_go_df['P_rho_mean_genie'], merged_go_df['P_rho_mean_cl'], 
                                         'System Importance\n(Clinical samples)', 'System Importance\n(Cell lines)')
print('Spearman rho:', stats.spearmanr(merged_go_df['P_rho_mean_cl'], merged_go_df['P_rho_mean_genie'])[0])

In [ ]:
scatterplot_sys_go.savefig('../plots/figure6/scatterplot_system_importance_go.svg')

In [ ]:
drug = 'Palbociclib'
fold_size = 5

for i in range(1, fold_size+1):
    create_gene_rho(drug, gene_list, '')
    create_gene_rho(drug, gene_list, '_genie_428')

In [ ]:
aggregate_gene_rho(drug, gene_list, '')
aggregate_gene_rho(drug, gene_list, '_genie_428')

In [ ]:
cell_gene_rho_df = pd.read_csv('../models/rlipp/' + drug + '_all_gene_rho.txt', sep='\t')[['Gene', 'Rho']]
genie_gene_rho_df = pd.read_csv('../models/rlipp/' + drug + '_all_gene_rho_genie_428.txt', sep='\t')[['Gene', 'Rho']]

merged_gene_rho_df = pd.merge(cell_gene_rho_df, genie_gene_rho_df, on=['Gene'], suffixes=('_cl', '_genie'))
merged_gene_rho_df.dropna(inplace=True)
scatterplot_gene_rho = create_scatter_plot_genes(merged_gene_rho_df['Rho_genie'], merged_gene_rho_df['Rho_cl'],
                                         'Gene Importance\n(Clinical samples)', 'Gene Importance\n(Cell lines)')
print('Spearman rho:', stats.spearmanr(merged_gene_rho_df['Rho_genie'], merged_gene_rho_df['Rho_cl'])[0])

In [ ]:
scatterplot_gene_rho.savefig('../plots/figure6/scatterplot_gene_rho_March_28.svg')

In [ ]:
mutations = pd.read_csv('../data/training_files_av/cell2mutation_ctg_av.txt', header=None, names=gene_list)
cn_deletions = pd.read_csv('../data/training_files_av/cell2cndeletion_ctg_av.txt', header=None, names=gene_list)
cn_amplifications = pd.read_csv('../data/training_files_av/cell2cnamplification_ctg_av.txt', header=None, names=gene_list)
cell_features = np.array(mutations | cn_deletions | cn_amplifications)

mutations = pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)
cn_deletions = pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)
cn_amplifications = pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)
genie_features = np.array(mutations | cn_deletions | cn_amplifications)

gene_sp = gene_importance(drug, cell_features, genie_features, gene_list, fold_size=5)

In [ ]:
gene_sp.savefig('../plots/figure6/scatterplot_gene_importance_March_25.svg')

In [ ]:
system_names = ['NEST:85', 'NEST:30', 'NEST:132']

drug = 'Palbociclib'

for s in system_names:

    test_data = pd.read_csv("../data/GENIE/test_428_" + drug + ".txt", sep='\t', header=None, names=['cell_line', 'smiles', 'auc', 'dataset'])
    
    genie_drug_data = genie_data.query("Sample_ID in @sample_list")
    genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
    genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
    genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
    print('Sample size:', len(genie_drug_data))
    genie_drug_data.reset_index(drop=True, inplace=True)

    _, km_fig_sys = genie_analysis_system(s, test_data, genie_drug_data, drug, resp_th=4)
    #km_fig_sys.savefig('../plots/figure6/survival_' + s + '.svg')

In [ ]:
#All Sens

drug = 'Palbociclib'
title = 'All Sensitive'
gene_feature_map = {'CREBBP':['m'], 'MLH1':['a'], 'MYC':['m'], 'CCNE1':['a']}

gene_alterations = np.zeros(len(sample_map), dtype=int)
for g, feature_list in gene_feature_map.items():
    for f in feature_list:
        if f == 'm':
            mutations = np.array(pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(mutations))
            gene_alterations |= mutations
        elif f == 'a':
            amplifications = np.array(pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(amplifications))
            gene_alterations |= amplifications
        else:
            deletions = np.array(pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(deletions))
            gene_alterations |= deletions
            
print(np.count_nonzero(gene_alterations))

genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_nest17_sens = genie_analysis_gene(gene_alterations, sample_map, genie_drug_data, title, is_resistant=False)

In [ ]:
#All Resistant

drug = 'Palbociclib'
title = 'All Resistant'
gene_feature_map = {'CREBBP':['d'], 'EP300':['d'], 
                    'CCNE1':['m','a'], 'BUB1B':['m','d'], 'RB1':['m','d'], 
                    'ERBB4':['d'], 'EGFR':['m','a'], 
                    'AR':['m','a'], 'KAT6A':['m','d'], 'TERT':['a','d'], 'RUNX1':['a']}

gene_alterations = np.zeros(len(sample_map), dtype=int)
for g, feature_list in gene_feature_map.items():
    for f in feature_list:
        if f == 'm':
            mutations = np.array(pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(mutations))
            gene_alterations |= mutations
        elif f == 'a':
            amplifications = np.array(pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(amplifications))
            gene_alterations |= amplifications
        else:
            deletions = np.array(pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(deletions))
            gene_alterations |= deletions
            
print(np.count_nonzero(gene_alterations))

genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_nest17_res = genie_analysis_gene(gene_alterations, sample_map, genie_drug_data, title, is_resistant=True)

In [ ]:
#NEST:17 Sensitive

drug = 'Palbociclib'
title = 'NEST:17 Sensitive'
gene_feature_map = {'CREBBP':['m'], 'CTCF':['a'], 'DDX3X':['m','a'], 'MDM2':['m']}

gene_alterations = np.zeros(len(sample_map), dtype=int)
for g, feature_list in gene_feature_map.items():
    for f in feature_list:
        if f == 'm':
            mutations = np.array(pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(mutations))
            gene_alterations |= mutations
        elif f == 'a':
            amplifications = np.array(pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(amplifications))
            gene_alterations |= amplifications
        else:
            deletions = np.array(pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(deletions))
            gene_alterations |= deletions
            
print(np.count_nonzero(gene_alterations))

genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_nest17_sens = genie_analysis_gene(gene_alterations, sample_map, genie_drug_data, title, is_resistant=False)

In [ ]:
#km_fig_palbo_rb1.savefig('../plots/figure6/survival_rb1_' + drug + '.svg')

In [ ]:
#NEST:17 Resistant

drug = 'Palbociclib'
title = 'NEST:17 Resistant'
gene_feature_map = {'CREBBP':['d'], 'EP300':['d'], 'CASP8':['d']}

gene_alterations = np.zeros(len(sample_map), dtype=int)
for g, feature_list in gene_feature_map.items():
    for f in feature_list:
        if f == 'm':
            mutations = np.array(pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(mutations))
            gene_alterations |= mutations
        elif f == 'a':
            amplifications = np.array(pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(amplifications))
            gene_alterations |= amplifications
        else:
            deletions = np.array(pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(deletions))
            gene_alterations |= deletions
            
print(np.count_nonzero(gene_alterations))

genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_nest17_res = genie_analysis_gene(gene_alterations, sample_map, genie_drug_data, title, is_resistant=True)

In [ ]:
#NEST:30 sensitive

drug = 'Palbociclib'
title = 'NEST:30 Sensitive'
gene_feature_map = {'MLH1':['a'], 'XRCC2':['d'], 'MLH3':['m', 'd']}

gene_alterations = np.zeros(len(sample_map), dtype=int)
for g, feature_list in gene_feature_map.items():
    for f in feature_list:
        if f == 'm':
            mutations = np.array(pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(mutations))
            gene_alterations |= mutations
        elif f == 'a':
            amplifications = np.array(pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(amplifications))
            gene_alterations |= amplifications
        else:
            deletions = np.array(pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(deletions))
            gene_alterations |= deletions
            
print(np.count_nonzero(gene_alterations))

genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_nest30_sens = genie_analysis_gene(gene_alterations, sample_map, genie_drug_data, title, is_resistant=False)

In [ ]:
#NEST:30 resistant

drug = 'Palbociclib'
title = 'NEST:30 Resistant'
gene_feature_map = {'CKS1B':['m'], 'CCNE1':['m','a'], 'BUB1B':['m','d'], 'RB1':['m','d']}

gene_alterations = np.zeros(len(sample_map), dtype=int)
for g, feature_list in gene_feature_map.items():
    for f in feature_list:
        if f == 'm':
            mutations = np.array(pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(mutations))
            gene_alterations |= mutations
        elif f == 'a':
            amplifications = np.array(pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(amplifications))
            gene_alterations |= amplifications
        else:
            deletions = np.array(pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(deletions))
            gene_alterations |= deletions
            
print(np.count_nonzero(gene_alterations))

genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_nest30_res = genie_analysis_gene(gene_alterations, sample_map, genie_drug_data, title, is_resistant=True)

In [ ]:
#NEST:132 sensitive

drug = 'Palbociclib'
title = 'NEST:132 Sensitive'
gene_feature_map = {'MYC':['m'], 'EGF':['a']}

gene_alterations = np.zeros(len(sample_map), dtype=int)
for g, feature_list in gene_feature_map.items():
    for f in feature_list:
        if f == 'm':
            mutations = np.array(pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(mutations))
            gene_alterations |= mutations
        elif f == 'a':
            amplifications = np.array(pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(amplifications))
            gene_alterations |= amplifications
        else:
            deletions = np.array(pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(deletions))
            gene_alterations |= deletions
            
print(np.count_nonzero(gene_alterations))

genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_nest30_sens = genie_analysis_gene(gene_alterations, sample_map, genie_drug_data, title, is_resistant=False)

In [ ]:
#NEST:132 resistant

drug = 'Palbociclib'
title = 'NEST:132 Resistant'
gene_feature_map = {'ERBB4':['d'], 'EGFR':['m','a'], 'RB1':['m','d']}

gene_alterations = np.zeros(len(sample_map), dtype=int)
for g, feature_list in gene_feature_map.items():
    for f in feature_list:
        if f == 'm':
            mutations = np.array(pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(mutations))
            gene_alterations |= mutations
        elif f == 'a':
            amplifications = np.array(pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(amplifications))
            gene_alterations |= amplifications
        else:
            deletions = np.array(pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(deletions))
            gene_alterations |= deletions
            
print(np.count_nonzero(gene_alterations))

genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_nest30_res = genie_analysis_gene(gene_alterations, sample_map, genie_drug_data, title, is_resistant=True)

In [ ]:
#NEST:85 sensitive

drug = 'Palbociclib'
title = 'NEST:85 Sensitive'
gene_feature_map = {'MYC':['m'], 'CREBBP':['m']}

gene_alterations = np.zeros(len(sample_map), dtype=int)
for g, feature_list in gene_feature_map.items():
    for f in feature_list:
        if f == 'm':
            mutations = np.array(pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(mutations))
            gene_alterations |= mutations
        elif f == 'a':
            amplifications = np.array(pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(amplifications))
            gene_alterations |= amplifications
        else:
            deletions = np.array(pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(deletions))
            gene_alterations |= deletions
            
print(np.count_nonzero(gene_alterations))

genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_nest30_sens = genie_analysis_gene(gene_alterations, sample_map, genie_drug_data, title, is_resistant=False)

In [ ]:
#NEST:85 resistant

drug = 'Palbociclib'
title = 'NEST:85 Resistant'
gene_feature_map = {'AR':['m','a'], 'KAT6A':['m','d'], 'TERT':['m','a','d'], 'RUNX1':['a']}

gene_alterations = np.zeros(len(sample_map), dtype=int)
for g, feature_list in gene_feature_map.items():
    for f in feature_list:
        if f == 'm':
            mutations = np.array(pd.read_csv('../data/GENIE/cell2mutation_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(mutations))
            gene_alterations |= mutations
        elif f == 'a':
            amplifications = np.array(pd.read_csv('../data/GENIE/cell2cnamplification_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(amplifications))
            gene_alterations |= amplifications
        else:
            deletions = np.array(pd.read_csv('../data/GENIE/cell2cndeletion_428.txt', header=None, names=gene_list)[g])
            print(g, f, np.count_nonzero(deletions))
            gene_alterations |= deletions
            
print(np.count_nonzero(gene_alterations))

genie_drug_data = genie_data.query("Sample_ID in @sample_list")
genie_drug_data = genie_drug_data[genie_drug_data[func_map[drug]] == 'Yes']
genie_drug_data = genie_drug_data[genie_drug_data['mTOR_Inhibitor_Overall'] == 'No']
genie_drug_data = genie_drug_data[genie_drug_data['AKT_Inhibitor_Overall'] == 'No']
print('Sample size:', len(genie_drug_data))
genie_drug_data.reset_index(drop=True, inplace=True)

survival_df, km_fig_nest30_res = genie_analysis_gene(gene_alterations, sample_map, genie_drug_data, title, is_resistant=True)

In [ ]:
#print(genie_drug_data.columns)